<a href="https://colab.research.google.com/github/orifelszer/CrimeData/blob/eden-branch/Supervised_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crimes Data in Israel (2019-2024)

#### by: Eden Shmuel, Oriana Feltzer

***

# Importing necessary libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import glob

In [2]:
# הורדת הקובץ מ-GitHub
!wget -O Data_Handling.py "https://raw.githubusercontent.com/orifelszer/CrimeData/main/Data_Handling.py"

# ייבוא הפונקציה מהקובץ
from Data_Handling import Preprocessing

--2025-01-05 16:14:45--  https://raw.githubusercontent.com/orifelszer/CrimeData/main/Data_Handling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5256 (5.1K) [text/plain]
Saving to: ‘Data_Handling.py’

Data_Handling.py    100%[===================>]   5.13K  --.-KB/s    in 0s      

2025-01-05 16:14:45 (50.5 MB/s) - ‘Data_Handling.py’ saved [5256/5256]



Define Paths to Crime Datasets (2019–2024)

In [3]:
!git clone https://github.com/orifelszer/CrimeData.git

import os
import zipfile

zip_folder = 'CrimeData'

zip_files = [f for f in os.listdir(zip_folder) if f.endswith('.zip')]

for zip_file in zip_files:
    zip_path = os.path.join(zip_folder, zip_file)
    extract_path = os.path.join(zip_folder, zip_file.replace('.zip', ''))

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Extracted: {zip_file} -> {extract_path}")

Cloning into 'CrimeData'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 106 (delta 27), reused 13 (delta 13), pack-reused 68 (from 1)
Receiving objects: 100% (106/106), 97.29 MiB | 12.38 MiB/s, done.
Resolving deltas: 100% (52/52), done.
Extracted: crimes2024.zip -> CrimeData/crimes2024
Extracted: crimes2019.zip -> CrimeData/crimes2019
Extracted: crimes2020.zip -> CrimeData/crimes2020
Extracted: crimes2023.zip -> CrimeData/crimes2023
Extracted: crimes2021.zip -> CrimeData/crimes2021
Extracted: crimes2022.zip -> CrimeData/crimes2022


In [4]:
csv_files = glob.glob("CrimeData/**/*.csv", recursive=True)
dataframes = {f"Crimes_{file[-8:-4]}": pd.read_csv(file) for file in csv_files}

Load Crime Data into Pandas DataFrames

Preview 2019 Crime Data

In [5]:
combined_data = pd.concat(dataframes, axis=0, ignore_index=True)

In [6]:
# הסרת שורות עם ערך -1 בעמודת StatisticGroupKod
combined_data = combined_data[combined_data['StatisticGroupKod'] != -1]

בדיקת חריגים לפני חלוקה לאימון ומבחן

In [7]:
# שמירת עמודות החיזוי
statistic_group = combined_data['StatisticGroup']
statistic_group_kod = combined_data['StatisticGroupKod']

# הסרת עמודות החיזוי מהנתונים
features = combined_data.drop(['StatisticGroup', 'StatisticGroupKod'], axis=1)

# חלוקה לסט אימון ובדיקה
X_train, X_test, y_train, y_test = train_test_split(
    features,
    statistic_group,  # משתמשים רק ב-StatisticGroupKod כעמודת חיזוי
    test_size=0.2,
    random_state=42,
    stratify=statistic_group  # שומר על התפלגות הקטגוריות
)

# שמירת עמודת StatisticGroup (לשימוש אפשרי בעתיד)
y_train_text = statistic_group_kod.loc[X_train.index]
y_test_text = statistic_group_kod.loc[X_test.index]

In [8]:
# ניקוי סט האימון ויצירת מיפויים
X_train_cleaned, train_mappings, scaler = Preprocessing(X_train, fit_scaler=True)

# ניקוי סט הבדיקה תוך שימוש במיפויים שנוצרו בסט האימון
X_test_cleaned, _, _ = Preprocessing(X_test, train_mappings=train_mappings, scaler=scaler)

In [9]:
# הגדרת עמודות לשמירה על בסיס סט האימון בלבד
columns_to_keep = [col for col in X_train_cleaned.columns if X_train_cleaned[col].sum() >= 10]

# שמירה על עמודות אימון רלוונטיות בלבד
X_train_cleaned = X_train_cleaned[columns_to_keep]

# התאמת סט המבחן רק לפי מה שנקבע באימון
for col in columns_to_keep:
    if col not in X_test_cleaned.columns:
        X_test_cleaned[col] = 0

# הבטחה ששני הסטים באותו סדר עמודות
X_test_cleaned = X_test_cleaned[X_train_cleaned.columns]

In [10]:
# נתונים לעבודה
X_train = X_train_cleaned
X_test = X_test_cleaned

In [11]:
# התאמת אינדקסים
y_train = y_train.loc[X_train_cleaned.index]
y_test = y_test.loc[X_test_cleaned.index]

Models

In [12]:
# Reducing memory usage
def optimize_data_types(df):
    for col in df.columns:
        col_type = df[col].dtype

        # אם מדובר במשתנה קטגוריאלי טקסטואלי
        if col_type == 'object':
            df[col] = df[col].astype('category')

        # אם מדובר במשתנה מספרי רציף
        elif col_type == 'float64':
            df[col] = df[col].astype('float32')

        # אם מדובר במשתנה מספרי שלם
        elif col_type == 'int64':
            df[col] = df[col].astype('int32')
    return df

# החלת הפונקציה על סט הנתונים
X_train = optimize_data_types(X_train)
X_test = optimize_data_types(X_test)


In [13]:
# ייבוא הספריות הדרושות
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from itertools import product
from sklearn.ensemble import RandomForestClassifier

In [14]:
# # דגימה של חלק מסט האימון בלבד
# X_train_sample = X_train.sample(frac=0.05, random_state=42)
# y_train_sample = y_train.loc[X_train_sample.index]

# # בחירת התכונות על סמך הדגימה בלבד
# selector = SelectKBest(score_func=chi2, k=20)
# selector.fit(X_train_sample, y_train_sample)

# # החלת הבחירה על כל סט האימון והבדיקה
# X_train_selected = selector.transform(X_train)
# X_test_selected = selector.transform(X_test)

In [ ]:
# # 📈 מודל בסיסי ללא שיפורים (Baseline)
# sgd_model = SGDClassifier(loss='log_loss', class_weight='balanced', max_iter=100, random_state=42)
# sgd_model.fit(X_train_selected, y_train)

# # 📊 חיזוי על סט הבדיקה
# y_pred_baseline = sgd_model.predict(X_test_selected)

# # ✅ הערכת ביצועי המודל
# accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
# print(f"Baseline Test Accuracy: {accuracy_baseline:.4f}")

In [ ]:
# # 🔁 קרוס-ולידציה על המודל הבסיסי
# cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
# cv_scores = cross_val_score(sgd_model, X_train_selected, y_train, cv=cv, scoring='accuracy')

# # תוצאות
# print(f"Cross-Validation Scores: {cv_scores}")
# print(f"Average Accuracy: {np.mean(cv_scores):.4f}")

In [15]:
# 📊 פונקציה למציאת הפרמטרים האופטימליים של SGDClassifier
def find_best_sgd_combination(X_train, y_train, X_test, y_test, alphas, max_iters, tols, k_features, cv_folds=3):
    best_score = 0
    best_params = {}
    results = []

    # לולאה על כל הקומבינציות האפשריות
    for alpha, max_iter, tol, k in product(alphas, max_iters, tols, k_features):

        # 🧩 בחירת התכונות
        selector = SelectKBest(score_func=chi2, k=k)
        selector.fit(X_train, y_train)
        X_train_selected = selector.transform(X_train)
        X_test_selected = selector.transform(X_test)

        # 📈 בניית מודל SGD עם הפרמטרים שנבדקים
        model = SGDClassifier(
            loss='log_loss',
            alpha=alpha,
            max_iter=max_iter,
            tol=tol,
            random_state=42,
            class_weight='balanced'
        )

        # 🔁 קרוס-ולידציה
        cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
        cv_scores = cross_val_score(model, X_train_selected, y_train, cv=cv, scoring='f1_macro')

        # אימון המודל על כל הדאטה
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)

        # 📊 חישוב מדדים
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='macro')
        recall = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')

        # שמירת התוצאות בכל הרצה
        results.append({
            'alpha': alpha,
            'max_iter': max_iter,
            'tol': tol,
            'features': k,
            'cv_f1_score': np.mean(cv_scores),
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

        # בדיקה אם התוצאה הנוכחית היא הטובה ביותר
        if f1 > best_score:
            best_score = f1
            best_params = {
                'alpha': alpha,
                'max_iter': max_iter,
                'tol': tol,
                'features': k,
                'f1': f1
            }

    #הדפסת התוצאה הטובה ביותר
    print(f"\n📊 Best Parameters Found:")
    print(f"Alpha: {best_params['alpha']}, Max Iter: {best_params['max_iter']}, Tol: {best_params['tol']}, Features: {best_params['features']}")
    print(f"Best F1-Score: {best_params['f1']:.4f}")

    #החזרת כל התוצאות והפרמטרים הטובים ביותר
    return results, best_params

In [ ]:
# דוגמה לפרמטרים אפשריים לבדיקה
alphas = [0.0001, 0.001, 0.01]
max_iters = [100, 300]
tols = [1e-3, 1e-4]
k_features = [10, 20]

# קריאה לפונקציה עם כל הקומבינציות
results, best_params = find_best_sgd_combination(X_train, y_train, X_test, y_test, alphas, max_iters, tols, k_features)

# הדפסת הקומבינציה הטובה ביותר
print("Best Parameters:", best_params)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# def find_best_random_forest_combination(X_train, y_train, X_test, y_test, n_estimators_list, max_depth_list, k_features, cv_folds=3):
#     best_score = 0
#     best_params = {}
#     results = []

#     # לולאה על כל הקומבינציות האפשריות
#     for n_estimators, max_depth, k in product(n_estimators_list, max_depth_list, k_features):

#         # 🧩 בחירת התכונות
#         selector = SelectKBest(score_func=chi2, k=k)
#         selector.fit(X_train, y_train)
#         X_train_selected = selector.transform(X_train)
#         X_test_selected = selector.transform(X_test)

#         # 📈 בניית מודל Random Forest עם הפרמטרים שנבדקים
#         model = RandomForestClassifier(
#             n_estimators=n_estimators,
#             max_depth=max_depth,
#             random_state=42,
#             class_weight='balanced')

#         # 🔁 קרוס-ולידציה
#         cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
#         cv_scores = cross_val_score(model, X_train_selected, y_train, cv=cv, scoring='f1_macro')

#         # אימון המודל על כל הדאטה
#         model.fit(X_train_selected, y_train)
#         y_pred = model.predict(X_test_selected)

#         # 📊 חישוב מדדים
#         accuracy = accuracy_score(y_test, y_pred)
#         precision = precision_score(y_test, y_pred, average='macro')
#         recall = recall_score(y_test, y_pred, average='macro')
#         f1 = f1_score(y_test, y_pred, average='macro')

#         # שמירת התוצאות בכל הרצה
#         results.append({
#             'n_estimators': n_estimators,
#             'max_depth': max_depth,
#             'features': k,
#             'cv_f1_score': np.mean(cv_scores),
#             'accuracy': accuracy,
#             'precision': precision,
#             'recall': recall,
#             'f1': f1})

#         # בדיקה אם התוצאה הנוכחית היא הטובה ביותר
#         if f1 > best_score:
#             best_score = f1
#             best_params = {
#                 'n_estimators': n_estimators,
#                 'max_depth': max_depth,
#                 'features': k,
#                 'f1': f1}

#     # ✅ הדפסת התוצאה הטובה ביותר
#     print(f"\n📊 Best Parameters Found:")
#     print(f"N Estimators: {best_params['n_estimators']}, Max Depth: {best_params['max_depth']}, Features: {best_params['features']}")
#     print(f"Best F1-Score: {best_params['f1']:.4f}")

#     # ✅ החזרת כל התוצאות והפרמטרים הטובים ביותר
#     return results, best_params

In [ ]:
# # דוגמה לפרמטרים אפשריים לבדיקה
# n_estimators_list = [50, 100, 200]
# max_depth_list = [10, 15, 20]  # None משמע ללא הגבלת עומק
# k_features = [10, 20, 30]

# # קריאה לפונקציה עם כל הקומבינציות
# results, best_params = find_best_random_forest_combination(X_train, y_train, X_test, y_test, n_estimators_list, max_depth_list, k_features)

# # הדפסת הקומבינציה הטובה ביותר
# print("Best Parameters:", best_params)

*****